<a href="https://colab.research.google.com/github/NicoG2023/Data_Science_Final_Project/blob/Predata/src/data-science-project/data_science_project/notebooks/EDA_Rain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA "Grupo Bimbo Inventory Demand"

### Introduction

Grupo Bimbo, a leading multinational bakery company, faces a unique challenge in managing the inventory of its products. With a typical shelf life of just one week, the accuracy of daily inventory calculations is paramount. Currently, these calculations are performed by direct delivery sales employees who rely on their personal experiences to predict the forces of supply, demand, and consumer behavior at each store. The margin for error in this process is minimal. Underestimating demand results in empty shelves and lost sales, while overestimating demand leads to excess product returns and increased expenses.

Grupo Bimbo aims to create a predictive model that can accurately forecast inventory needs based on historical data, thereby optimizing the supply chain and improving efficiency.


## Libraries

In [ ]:
# importing the basic libraries
#!pip install ydata_profiling
# !pip install catboost
# !pip install lightgbm
# !pip install xgboost
# !pip install plotly
# !pip install path
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.metrics import roc_auc_score, accuracy_score
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import accuracy_score
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier
# import path
import os
# import plotly.express as px
import gc

## Charge data

In [ ]:
train_df = pd.read_csv("../Data/train/train.csv",nrows=1000)
client_df = pd.read_csv("../Data/cliente_tabla.csv")
products_df = pd.read_csv("../Data/producto_tabla.csv")
test_df = pd.read_csv("../Data/test/test.csv")
town_state_df = pd.read_csv("../Data/town_state.csv")

In [ ]:
# Inspect the data, with funcion

def inspect_columns(df:pd.DataFrame, df_name:str)->None:
    """
    A helper function that does a deep analisis about the columns in a dataframe
    Inputs:
     df(pd.Dataframe): the dataframe that will be inspected
     df_name(str): the tittle that will be put in the output(print) of the function
    Returns:
     None
    """

    total_rows = len(df)
    result = pd.DataFrame({
        'total_rows': [total_rows] * df.shape[1],
        'rows_with_missing_values': df.isnull().sum(),
        'unique': df.nunique() == total_rows,
        'cardinality': df.nunique(),
        'with_null': df.isna().any(),
        'null_pct': round((df.isnull().sum() / total_rows) * 100, 2),
        '1st_row': df.iloc[0],
        'random_row': df.iloc[np.random.randint(low=0, high=total_rows)],
        'last_row': df.iloc[-1],
        'dtype': df.dtypes,
    })

    # Print the name of the dataframe
    print(f"\n{'='*10} {df_name} {'='*10}\n")

    # Print the head of the dataframe
    print("First few rows of the dataframe:\n")
    display(df.head())

    # Print the resulting statistics
    print("Detailed statistics:\n")
    display(result)

## Understand the data

We performed a preliminary exploration to understand the structure of the data.

In [ ]:
inspect_columns(train_df,"Training Dataframe")
inspect_columns(client_df,"Client Dataframe")
inspect_columns(products_df,"Products Dataframe")
inspect_columns(test_df,"Test Dataframe")
inspect_columns(town_state_df,"Town and State Dataframe")

### Exploratory data analysis


In [ ]:
# transform by merge with the data

train_df = train_df.merge(client_df,on = "Cliente_ID",how="left")
train_df = train_df.merge(products_df,on = "Producto_ID",how="left")
train_df = train_df.merge(town_state_df,on = "Agencia_ID",how="left")

train_df.head()

In [ ]:
# translate the language used for the data

train_df = train_df.rename({'Semana':'Week', 'Agencia_ID':'Sales_Depot_ID', 'Canal_ID':'Sales_Channel_ID', 'Ruta_SAK':'Route_ID', 'Cliente_ID':'Client_ID',
       'Producto_ID':'Product_ID', 'Venta_uni_hoy':'Sales_unit_this_week ', 'Venta_hoy':'Sales_this_week', 'Dev_uni_proxima':'Returns_unit_next_week',
       'Dev_proxima':'Returns_next_week', 'Demanda_uni_equil':'Adjusted_Demand', 'NombreCliente':'Client_name', 'NombreProducto':'Product_Name',
       'Town':'Town', 'State':'State'},axis=1)

train_df.head()

In [ ]:
# inspect the data with ydata profile
sample_df = train_df.sample(frac=0.2, random_state=1)
profile_obj = ProfileReport(sample_df, title='Bimbo Inventory Demand Data Profiling Report', minimal=True)
profile_obj.to_file('../Data/html-files/train_final_minimal.html')
profile_obj

In [ ]:
profile_obj = ProfileReport(sample_df, title='Bimbo Inventory Demand Data Profiling Report')
profile_obj.to_file('../Data/html-files/train_final_normal.html')
profile_obj

In [ ]:
sample_df = train_df.sample(frac=0.1, random_state=1)
profile_obj = ProfileReport(sample_df, title='Bimbo Inventory Demand Data Profiling Report', explorative=True)
profile_obj.to_file('../Data/html-files/train_final_explorative.html')
profile_obj

# Web Scraping

According to the dataset, the data was taken in a timespan of 9 weeks. It's been decided to get the biweekly inflation and the consumer confidence index from Mexico. The info was taken from March 31st, 2016, until June 1st, 2016.

In [ ]:
#Getting biweekly inflation
import sys
import requests
from datetime import datetime

api_key = '74bcad49-835f-fab1-c9f0-fdab92570186'

api_url = f'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/628229/es/0700/false/BIE/2.0/{api_key}?type=json'

response = requests.get(api_url)
if response.status_code == 200:
    inflation_data = response.json()
else:
    print('ERROR FAILED TO RETRIEVE DATA')
    sys.exit()

observations = inflation_data['Series'][0]['OBSERVATIONS']
dates = []
values = []
for obs in observations:
    dates.append(obs['TIME_PERIOD'])
    values.append(obs['OBS_VALUE'])

inflation_df = pd.DataFrame({
    'Fecha': dates,
    'Inflacion': values
})

inflation_df['Fecha'] = pd.to_datetime(inflation_df['Fecha'], format='%Y/%m/%d')
start_date = datetime(2016, 3, 31)
end_date = datetime(2016, 6, 1)

filtered_inflation_df = inflation_df[(inflation_df['Fecha'] >= start_date) & (inflation_df['Fecha'] <= end_date)]
filtered_inflation_df = filtered_inflation_df.reset_index(drop=True)
filtered_inflation_df

In [ ]:
#Getting consumer confidence index
api_key = '74bcad49-835f-fab1-c9f0-fdab92570186'
api_url = f'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/454168/es/0700/false/BIE/2.0/{api_key}?type=json'

response = requests.get(api_url)
if response.status_code == 200:
    confidence_data = response.json()
else:
    print('ERROR FAILED TO RETRIEVE DATA')
    sys.exit()

observations = confidence_data['Series'][0]['OBSERVATIONS']
dates = []
values = []

for obs in observations:
    dates.append(obs['TIME_PERIOD'])
    values.append(obs['OBS_VALUE'])

confidence_data = pd.DataFrame({
    'Fecha': dates,
    'Consumer_confidence': values
})

confidence_data['Fecha'] = pd.to_datetime(confidence_data['Fecha'], format='%Y/%m')
start_date = datetime(2016, 3, 31)
end_date = datetime(2016, 6, 1)

filtered_confidence_df = confidence_data[(confidence_data['Fecha'] >= start_date) & (confidence_data['Fecha'] <= end_date)]
filtered_confidence_df = filtered_confidence_df.reset_index(drop=True)
filtered_confidence_df

## Data Merging
Having charged and understood the data now we proceed to merge it in order to create a consise and rich dataset for the model to be given 

In [106]:
# merging train_df, filtered_confidence_df, filtered_inflation_df
webs_df = pd.merge(filtered_confidence_df,filtered_inflation_df,on="Fecha",how="outer")
final_df = pd.concat([train_df,webs_df],axis=1)

print(final_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Week                    1025 non-null   int64         
 1   Sales_Depot_ID          1025 non-null   int64         
 2   Sales_Channel_ID        1025 non-null   int64         
 3   Route_ID                1025 non-null   int64         
 4   Client_ID               1025 non-null   int64         
 5   Product_ID              1025 non-null   int64         
 6   Sales_unit_this_week    1025 non-null   int64         
 7   Sales_this_week         1025 non-null   float64       
 8   Returns_unit_next_week  1025 non-null   int64         
 9   Returns_next_week       1025 non-null   float64       
 10  Adjusted_Demand         1025 non-null   int64         
 11  Client_name             1025 non-null   object        
 12  Product_Name            1025 non-null   object  

In [ ]:
plt.figure(figsize=(20, 20))  # Adjust figure size as needed
plt.barh(final_df["Product_Name"], final_df["Sales_this_week"])
plt.title("Sales per Product")
plt.xlabel("Sales")
plt.ylabel("Product Name")
plt.show()

In [ ]:
sns.lmplot(x="Sales_this_week",y="Returns_next_week",data=final_df)
plt.title("Expected Demand")

In [ ]:
plt.figure(figsize=(20,20))
plt.scatter(final_df["Sales_this_week"],final_df["Client_name"],color="r",label="Sales")
plt.scatter(final_df["Returns_next_week"],final_df["Client_name"],color="b",label="Refund")
plt.legend()
plt.title("Sales and Refunds per Client")

In [ ]:
plt.plot(final_df[])

In [ ]:
plt.figure(figsize=(20,20))
plt.scatter(final_df["Sales_this_week"],final_df["Client_name"],color="r",label="Sales")
plt.scatter(final_df["Returns_next_week"],final_df["Client_name"],color="b",label="Refund")
plt.legend()
plt.title("Sales and Refunds per Client")

In [ ]:
plt.figure(figsize=(20, 20))  # Adjust figure size as needed
plt.barh(final_df["Product_Name"], final_df["Sales_this_week"])
plt.title("Sales per Product")
plt.xlabel("Sales")
plt.ylabel("Product Name")
plt.show()